# Run: pmito5 - Mitovirus/Narnavirus pilot

```
Lead     : ababaian
Issue    : 
Version  : v0.3.5-dev : diamond-dev branch
start    : 2020 10 25
complete : 2020 10 26
files    : ~/serratus/notebook/201012_ab/
s3_files : s3://serratus-public/notebook/201012_ab/
output   : s3://serratus-public/out/201025_pmito5/
```

### Intro/Objectives

- Run protref5b against some test human RNAseq and ~1,000 random viromes for pilot


In [ ]:
# Fire up EC2 Instance
sudo yum install -y docker
sudo yum install -y git
sudo service docker start

# Download latest serratus repo
git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

# If you want to upload containers to your repository, include this.
export DOCKERHUB_USER='serratusbio' # optional
sudo docker login # optional

# Build all containers and upload them docker hub repo (if available)
./build_containers.sh

## Virome + Metatranscriptome

Query: `"VIRAL METAGENOME" OR "VIROME" OR "VIROMIC" OR "VIRAL RNA" OR "METATRANSCRIPTOMIC" NOT "METAGENOMIC" NOT amplicon[All Fields] AND "platform illumina"[Properties] AND cluster_public[prop]`

Date: `2000904`
Return: `60327`

Sub-sample to 1000 datasets

## Human test data

Query: `SRP163252`

Date: `201025`
Return: `165`


Saved to "$WORK"

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/201012_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/201012_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sun Oct 25 19:49:27 PDT 2020
fde8446fbf106afff5ec1c82941f154fc39c97f7


In [2]:
# Randomly select 1000 viro samples
head -n1 viro_SraRunInfo.csv > sra.header

# Inverse select viro and vert
shuf viro_SraRunInfo.csv | head -n 1000 > viro1k.tmp

cat sra.header viro1k.tmp \
  > viro1k_SraRunInfo.csv

rm viro1k.tmp
wc -l *.csv
md5sum *.csv

shuf: write error: Broken pipe
shuf: write error
     167 hutest_SraRunInfo.csv
    1001 viro1k_SraRunInfo.csv
   61466 viro_SraRunInfo.csv
   62634 total
26d72d550701deeeb578ca8399da8629  hutest_SraRunInfo.csv
c16a1b2da03eaf1088933a1de329ce2f  viro1k_SraRunInfo.csv
2ddb64e9f5706fd64354a45f296592b4  viro_SraRunInfo.csv


In [ ]:
# aws s3 sync ./ $S3_WORK

### Terraform Initialize

In [3]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..33bf4e8 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@ re

module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.monitoring.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_region.current: Refreshing state...
module.monitoring.module.ecs_cluster.data.aws_ami.ecs: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.merge.data.aws_region.current: Refreshing state...
module.scheduler.module.ecs_cluster.data.aws_ami.ecs: Refreshing state...
module.download.data.aws_region.current: Refreshing state...
module.merge.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_availability_zones.all: Refreshing state...
module.align.data.aws_ami.amazon_linux_2: Refreshing state...
module.scheduler.random_password.pg_password: Creating...
module.scheduler.random_password.pg_password: Creation complete after 0s [id=none]
local_file.upload_sra: Creating...
local_file.upload_sra: Creation complete after 0s

module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-dl:CloudWatchLogsCreate-serratus-dl]
module.download.aws_iam_role_policy.ec2Describe: Creating...
module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-dl-20201026031446137100000006]
module.download.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-merge-20201026031446223300000007]
module.download.aws_iam_role_policy.ec2Terminate: Creating...
aws_security_group.internal: Creation complete after 5s [id=sg-05d3672e5be6286b6]
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-ser

module.align.aws_autoscaling_group.worker: Creation complete after 1s [id=serratus-align-2020102603151494160000000a]
module.download.aws_autoscaling_policy.worker: Creating...
module.merge.aws_autoscaling_group.worker: Creation complete after 1s [id=serratus-merge-2020102603151495760000000c]
module.align.aws_autoscaling_policy.worker: Creating...
module.merge.aws_autoscaling_policy.worker: Creating...
module.align.aws_autoscaling_policy.worker: Creation complete after 1s [id=serratus-align-2020102603151494160000000a]
module.download.aws_autoscaling_policy.worker: Creation complete after 1s [id=serratus-dl-2020102603151494160000000b]
module.merge.aws_autoscaling_policy.worker: Creation complete after 2s [id=serratus-merge-2020102603151495760000000c]

Apply complete! Resources: 71 added, 0 changed, 0 destroyed.

Outputs:

align_asg_name = serratus-align-2020102603151494160000000a
dl_asg_name = serratus-dl-2020102603151494160000000b
help = Run ./create_tunnels.sh to create SSH tunnels for

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.0215,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 20,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 10,
  "GENOME": "protref5b",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 20,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [4]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

bind [127.0.0.1]:5432: Address already in use
Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [5]:
cd $WORK
BATCH='hutest_SraRunInfo.csv'
wc -l $WORK/$BATCH

167 /home/artem/serratus/notebook/201012_ab/hutest_SraRunInfo.csv


In [7]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/201012_ab/hutest_SraRunInfo.csv
  date: Sun Oct 25 20:18:35 PDT 2020
  wc  : 167 /home/artem/serratus/notebook/201012_ab/hutest_SraRunInfo.csv
  md5 : 26d72d550701deeeb578ca8399da8629  /home/artem/serratus/notebook/201012_ab/hutest_SraRunInfo.csv


--------------------------
tmp.chunk00
167 tmp.chunk00_sraRunInfo.csv
a8d3ed0c9f1f568a5a6c579aafb456ca  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":165,"total_rows":165}


 uploadSRA complete.


In [10]:
cd $WORK
BATCH='viro1k_SraRunInfo.csv'
wc -l $WORK/$BATCH

1001 /home/artem/serratus/notebook/201012_ab/viro1k_SraRunInfo.csv


In [11]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/201012_ab/viro1k_SraRunInfo.csv
  date: Sun Oct 25 21:06:48 PDT 2020
  wc  : 1001 /home/artem/serratus/notebook/201012_ab/viro1k_SraRunInfo.csv
  md5 : c16a1b2da03eaf1088933a1de329ce2f  /home/artem/serratus/notebook/201012_ab/viro1k_SraRunInfo.csv


--------------------------
tmp.chunk00
1001 tmp.chunk00_sraRunInfo.csv
06499ae61832717317e82b73b364cd86  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":1000,"total_rows":1165}


 uploadSRA complete.


## Run Serratus

In [12]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.1,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 500,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 300,
  "GENOME": "protref5b",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 20,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":25,"ALIGN_SCALING_CONSTANT":0.1,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":500,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_M

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
cd $TF

aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [15]:
cd $WORK

aws s3 cp hutest_SraRunInfo.csv s3://serratus-public/out/201025_pmito5/
aws s3 cp viro1k_SraRunInfo.csv s3://serratus-public/out/201025_pmito5/

upload: ./hutest_SraRunInfo.csv to s3://serratus-public/out/201025_pmito5/hutest_SraRunInfo.csv
upload: ./viro1k_SraRunInfo.csv to s3://serratus-public/out/201025_pmito5/viro1k_SraRunInfo.csv
